In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
im = np.copy(image)
def slope_lines(image,lines):
    
    img = image.copy()
    poly_vertices = []
    order = [0,1,3,2]

    left_lines = [] # Like /
    right_lines = [] # Like \
    for line in lines:
        for x1,y1,x2,y2 in line:

            if x1 == x2:
                pass #Vertical Lines
            else:
                m = (y2 - y1) / (x2 - x1)
                c = y1 - m * x1

                if m < 0:
                    left_lines.append((m,c))
                elif m >= 0:
                    right_lines.append((m,c))

    left_line = np.mean(left_lines, axis=0)
    right_line = np.mean(right_lines, axis=0)

    #print(left_line, right_line)

    for slope, intercept in [left_line, right_line]:

        #getting complete height of image in y1
        rows, cols = image.shape[:2]
        y1= int(rows) #image.shape[0]

        #taking y2 upto 60% of actual height or 60% of y1
        y2= int(rows*0.6) #int(0.6*y1)

        #we know that equation of line is y=mx +c so we can write it x=(y-c)/m
        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)
        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), [255, 0, 0], 10)
                #draw_lines(img, np.array([[[x1,y1,x2,y2]]]))
    
    poly_vertices = [poly_vertices[i] for i in order]
    cv2.fillPoly(img, pts = np.array([poly_vertices],'int32'), color = (0,255,0))
    return cv2.addWeighted(image,0.7,img,0.4,0.)

def lane_det(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Gaussian smoothing
    kernel_size = 7
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Canny Edge Detection
    low_threshold = 70
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Masked Edges
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255

    # Area of Interest
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450,320), (520,320), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Hough Transform
    rho = 2 
    theta = np.pi/180 
    threshold = 25     
    min_line_length = 5 
    #max_line_gap = 250   
    max_line_gap = 40
    line_image = np.copy(image)*0
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    print(lines)
    
    
    # Draw Lines
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
        
    # Create a "color" binary image to combine with line image
    #color_edges = np.dstack((edges, edges, edges))
    color_edges = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    color_edges = slope_lines(color_edges,lines)
    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(color_edges, 0.8,line_image, 1, 0)

    #Draw Lines on original image
    lines_edges1 = cv2.addWeighted(im,0.8,line_image,1,0)

    return lines_edges1

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
result = lane_det(image)
plt.imshow(result)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')


This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
[[[504 320 853 538]]

 [[498 320 787 508]]

 [[765 495 831 537]]

 [[310 422 448 329]]

 [[327 420 439 339]]

 [[321 424 433 340]]

 [[637 404 853 539]]

 [[182 521 189 516]]

 [[736 476 799 516]]

 [[358 392 458 320]]

 [[360 392 449 330]]

 [[506 320 522 330]]

 [[797 516 832 538]]

 [[390 373 448 332]]

 [[310 425 386 370]]

 [[698 451 725 468]]]


NameError: name 'lines' is not defined